## Imports and Vairable definitions

In [1]:
import os
import tensorflow as tf
# import keras
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

print(tf.__version__)

2.0.0


In [2]:
IMG_DIR = '/Users/jthomas/Code/DataScienceSIG/furniture-detector/img/'
TRAIN_DIR = IMG_DIR + 'train/'
VAL_DIR =  IMG_DIR + 'val/'
MODEL_DIR = '/Users/jthomas/Code/DataScienceSIG/saved-models/base_keras.h5'

IMAGE_SIZE = 224
BATCH_SIZE = 10

CLASS_INDICES = {'bed': 0, 'chair': 1, 'sofa': 2, 'swivelchair': 3, 'table': 4}

## Load model and predict
Load the saved model from the .h5 file and then predict results for the validation image set

In [3]:
model = tf.keras.models.load_model(MODEL_DIR)

datagen = tf.keras.preprocessing.image.ImageDataGenerator()

val_generator = datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=False)

CLASS_INDICES = val_generator.class_indices # Update class indexes based on results of generator

predictions = model.predict(
    val_generator,
    use_multiprocessing=True
) 

Found 423 images belonging to 5 classes.


In [4]:
print(val_generator.class_indices)

{'bed': 0, 'chair': 1, 'sofa': 2, 'swivelchair': 3, 'table': 4}


## Visualize input images
View a sample of the images that are being fed in as validation data

In [5]:
val_image_names = os.listdir(VAL_DIR)

plt.figure(figsize=(10,10))
for key, val in CLASS_INDICES.items():
    image_names = os.listdir(VAL_DIR + key)
    for i in range(5):
        plt.subplot(5, 5, (i*5)+val+1) #val is 0-4 so it functions as an 0-based itterator
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)

        img = Image.open(os.path.join(VAL_DIR,key,image_names[i]))
        plt.imshow(img)
        plt.xlabel(key)
plt.show()

## Plot the results of the validation test

In [0]:
def plot_image(class_labels, predictions_array, true_label, img_path):
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  img = Image.open(img_path)
  plt.imshow(img)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_labels[predicted_label],
                                100*np.max(predictions_array),
                                class_labels[true_label]),
                                color=color)


def plot_value_array(class_labels, predictions_array, true_label):
  num_classes = class_labels.items().count
  plt.grid(False)
  plt.xticks(range(num_classes), class_labels.keys())
  plt.yticks([])
  thisplot = plt.bar(range(num_classes), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')


# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 10
num_cols = 2
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  predictions = predictions[i]
  true_label = val_generator.classes[i]
  img_path = val_generator.filepaths[i]

  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(CLASS_INDICES, predictions, true_label, img_path)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(CLASS_INDICES, predictions, true_label)
plt.tight_layout()
plt.show()